# Django 6일차

Django authentication system(인증시스템)

- Django authentication system(인증시스템)은 인증(Authentication)과 권한(Authorization) 부여를 함께 제공(처리)하며, 이러한 기능을 일반적으로 인증 시스템이라고 함

- 필수 구성은 settings.py에 이미 포함되어 있으며 INSTALLED_APPS에서 확인 가능
    - django.contrib.auth

Authentication(인증)
- 신원 확인
- 사용자가 자신이 누구인지 확인하는 것

Authorization(권한, 허가)
- 권한 부여
- 인증된 사용자가 수행할 수 있는 작업을 결정

두번 째 app accounts 생성 및 등록
```
    python manage.py startapp accounts
```

```python
    # settings.py
    INSTALLED_APP = [
        'articles',
        'accounts',
    ]
```

url 분리 및 매핑
```python
    # accounts/urls.py
    from django.urls import path
    from . import views

    app_name = 'accounts'
    urlpatterns=[

    ]
```

```python
    # crud/urls.py
    urlpatters = [
        path('accounts/',include('accounts.urls')),
    ]
```


Custom User Model로 `대체`하기

기본 User Model을 필수적으로 Custom User model로 대체하는 이유 이해하기

Django는 기본적인 인증 시스템과 여러 가지 필드가 포함된 User Model을 제공, 대부분의 개발 환경에서 기본 User Model을 Custom User Model로 대체

# 쿠키와 세션의 차이점 !

* 쿠키 - 사이트 방문할때 브라우저에 저장되는 내용들 -> 서버가 아닌!! 클라이언트가 기록정보 data를 가지고 있음
       - 쿠키내용을 내가 수정할 수 있고 남이 훔쳐볼수도 있음 -> 장바구니 처럼 보안이 중요하지 않는경우/ 팝업창 오늘하루안보기

* 세션 - 클라이언트가 아닌!! 서버가 기록정보 data를 가지고 있음 - 비번 카드정보 처럼 기업이 책임져야 할 것들은 서버에 저장
         1. 사이트 접속시 서버가 기한이 짧은 "임시키"를 클라이언트에게 발급해서 클라이언트의 브라우저에 저장됨
         2. 그 다음부터 브라우저가 해상 웹사이트를 접속하면 http요청에 (플러스) 그전에 발급받았던 키 까지 더해서 요청메시지 전송
         3. 서버에서는 키를 확인 후 해당 클라이언트에게 정보를 제공 (즉, 세션은 서버가 클라이언트가 누구인지를 식별,알아보는 방법)
         4. 그러나 세션을 남발하면 접속자가 많을 시 서버에 부하가 걸릴 수 있음 -> 그래서 토큰방식으로 로그인 가능 ! 
            서버로 부터 토큰을 발급받은 클라이언트가 이를 쿠키로 저장해 두고 필요할 때마다 보여줌 -> 서버의 부하를 줄일 수 있음

로그인 - 새로운 세션을 하나 만든다


# 데이터베이스 초기화 (프로젝트중간에 하던것이 있을 경우)

1. migration파일 삭제 (init은 남겨놓음)
2. db.sqlite3 삭제
3. makemigration / migrate 실행

python manage.py makemigrations
python manage.py migrate
python manage.py createsuperuser
python manage.py runserver

4. 바뀐 테이블 확인 또는 
   관리자 계정 만든 후 로그인 되는지 확인하기
    로컬호스트 admin 들어가서 확인

-----------------------------------------------------------------

# 해당앱 template 작성 전 base.html 부터 보자

<body>
    <div class="container">
        <h3>Hello, {{ user }}</h3>
     
        <a href="{% url 'accounts:login' %}">Login</a>
    </div>     
    {% block content %}{% endblock content %}

</body>

-----------------------------------------------------------------

# accounts/templates/login.py 작성해보기

{% extends 'base.html' %}

{% block content %}
<h1>로그인</h1>
<form action="{% url 'accounts:login' %}" method="POST">

    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit">
</form>
{% endblock content %}

-----------------------------------------------------------------


# settings 에 추가하기

INSTALLED_APPS = [
    'accounts',

AUTH_USER_MODEL = 'accounts.User'  

# project/urls.py 에 추가하기

    path('accounts/', include('accounts.urls'))

기본셋팅 끝
-----------------------------------------------------------------
model -> admin -> url -> view (+forms) -> template 순으로 작성해 보자
-----------------------------------------------------------------


# accounts/models.py 에 유저 클래스 상속받아 로그인 관련 클래스 정의하기

from django.contrib.auth.models import AbstractUser

class User(AbstractUser):
    pass

-----------------------------------------------------------------

# accounts/admin.py 에 admin사이트에 user모델 등록

from django.contrib import admin
from django.contrib.auth.admin import UserAdmin
from .models import User

admin.site.register(User, UserAdmin)

-----------------------------------------------------------------

# accounts/urls.py 에 추가하기

from django.urls import path
from . import views

app_name = 'accounts'
urlpatterns = [
    path('login/', views.login, name='login'),
]

-----------------------------------------------------------------

# accounts/forms 만들기

from django.contrib.auth.forms import UserCreationForm
from django.contrib.auth import get_user_model

class CustomUserCreationForm(UserCreationForm):
    class Meta:
        model = get_user_model()
        fields = ('username', 'email',)

-----------------------------------------------------------------

# accounts/views.py 에 추가하기

from django.shortcuts import render, redirect
from django.contrib.auth import login as auth_login
from django.contrib.auth.forms import AuthenticationForm

def login(request):

    if request.method == 'POST':
        form = AuthenticationForm(request, request.POST)
        if form.is_valid():
            auth_login(request, form.get_user())
            return redirect('articles:index')
    else:
        form = AuthenticationForm()
    context = {
        'form': form,
    }
    return render(request, 'accounts/login.html', context)

-----------------------------------------------------------------

# 데이터베이스 초기화 (프로젝트중간에 하던것이 있을 경우)

1. migration파일 삭제 (init은 남겨놓음)
2. db.sqlite3 삭제
3. makemigration / migrate 실행

python manage.py makemigrations
python manage.py migrate
python manage.py createsuperuser
python manage.py runserver

4. 바뀐 테이블 확인 또는 
   관리자 계정 만든 후 로그인 되는지 확인하기
    로컬호스트 admin 들어가서 확인

-----------------------------------------------------------------